In [1]:
bs = 256
c=512

# Pytorch

In [2]:
import torch
import torch.nn as nn

In [3]:
x=  torch.rand(bs,c).cuda()
x.requires_grad=True
W = torch.rand(bs,c,c).cuda()
W.requires_grad=True
W2 = torch.rand(c,c).cuda()
W2.requries_grad=True

In [4]:
%timeit torch.cuda.synchronize();(W@x.unsqueeze(-1)).squeeze(-1).mean().backward();torch.cuda.synchronize()

4.2 ms ± 1.07 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [5]:
%timeit torch.cuda.synchronize();(W2@x.unsqueeze(-1)).squeeze(-1).mean().backward();torch.cuda.synchronize()

155 µs ± 2.47 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


# Jax

In [6]:
import jax.numpy as jnp
from jax import grad, jit, vmap
from jax import random
from jax import jacfwd, jacrev
import numpy as np
from functools import partial
from jax.ops import index, index_add, index_update

In [7]:
key = random.PRNGKey(0)
x = random.normal(key,(bs,c))
W = random.normal(key,(bs,c,c))
W2 = random.normal(key,(c,c))

In [8]:
@jit
def f(W):
    return (W@x[:,:,None]).mean()
g = jit(grad(f))
g(W);
g(W2);

In [9]:
%timeit g(W)

632 µs ± 46.5 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [10]:
%timeit g(W2)

92.7 µs ± 570 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
